### BASICALLY THE TRAINING CODE BUT ONLY THE VALIDATION PART

In [1]:
import base64

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2

import io
from PIL import Image
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import timm
import torch.nn as nn

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    balanced_accuracy_score,
    confusion_matrix,
    roc_auc_score
)

from sklearn.metrics import precision_recall_curve

from sklearn.metrics import precision_score, recall_score, fbeta_score


import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
import timm
import torch.nn as nn


In [2]:
df_tab_results = pd.read_csv("/kaggle/input/new-datasets/tabular_preds.csv")
df_tab_results.head()

,Unnamed: 0,lgbm_42,lgbm_52,lgbm_62,lgbm_72,lgbm_82,ensemble,target,isic_id
0,0,0.015197,0.012859,0.018430,0.014540,0.013881,0.014981,0,ISIC_0015670
1,1,0.994896,0.994877,0.994858,0.994904,0.994728,0.994852,0,ISIC_0015845
2,2,0.009457,0.008784,0.008883,0.007815,0.008454,0.008679,0,ISIC_0015864
3,3,0.017314,0.013926,0.014249,0.014940,0.013991,0.014884,0,ISIC_0015902
4,4,0.203507,0.222469,0.274706,0.196490,0.217909,0.223016,0,ISIC_0024200


In [3]:
df_images = pd.read_pickle("/kaggle/input/new-datasets/images_w.pkl")
df_images.head()

,isic_id,jpeg_bytes,labels
0,ISIC_0015670,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,0
1,ISIC_0015845,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,0
2,ISIC_0015864,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,0
3,ISIC_0015902,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,0
4,ISIC_0024200,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,0


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
# We apply this transform on the images before training

image_size = 224

data_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(p=0.5),  # simple augmentations
    transforms.RandomVerticalFlip(p=0.2),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet mean
        std=[0.229, 0.224, 0.225]    # ImageNet std
    ),
])


In [6]:
# Helps use the images (in jpeg bytes format in the df)

class JpegBytesDataset(Dataset):
    def __init__(self, df, image_col='jpeg_bytes', label_col='labels', id_col='isic_id', transform=None):
        """
        Args:
            df (pd.DataFrame): DataFrame containing the data.
            image_col (str): Name of the column with image bytes (or base64 strings).
            label_col (str): Name of the column with labels.
            id_col (str): Name of the column with unique sample IDs.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.df = df.reset_index(drop=True)
        self.image_col = image_col
        self.label_col = label_col
        self.id_col = id_col  # Added id_col attribute
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        jpeg_bytes = self.df.iloc[idx][self.image_col]

        if isinstance(jpeg_bytes, str):
            image_data = base64.b64decode(jpeg_bytes)
        else:
            image_data = jpeg_bytes

        image = Image.open(io.BytesIO(image_data)).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.df.iloc[idx][self.label_col]

        item_id = self.df.iloc[idx][self.id_col]

        return image, label, item_id

In [7]:
# Define model architecture
model = timm.create_model("eva02_base_patch14_224.mim_in22k", pretrained=False) # Use pretrained=False if loading a full checkpoint

# Freeze all parameters initially
for param in model.parameters():
    param.requires_grad = False

# Replace the 'head' (classifier) with a new layer
in_features = model.num_features
model.head = nn.Linear(in_features, 2)

# Unfreeze the layers to train
for param in model.head.parameters():
    param.requires_grad = True

if hasattr(model, "blocks") and len(model.blocks) >= 2:
    for param in model.blocks[-2].parameters():
        param.requires_grad = True

# Load saved weights
WEIGHTS_PATH = '/kaggle/input/final_img_models/tensorflow2/default/1/eva02_final2.pth' # <--- IMPORTANT: Change this to your file path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load(WEIGHTS_PATH, map_location=device)

model.load_state_dict(checkpoint, strict=False)

model.eval()

print("Custom weights loaded successfully!")

Custom weights loaded successfully!


In [8]:
model = model.to(device)

In [9]:
val_dataset = JpegBytesDataset(df_images, transform=data_transforms)

val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:

model.eval()
all_preds = []
all_ids = []

with torch.no_grad():
    for inputs, _, ids in tqdm(val_loader, desc="Generating Predictions"):

        inputs = inputs.to(device)

        outputs = model(inputs)

        probs = torch.softmax(outputs, dim=1)[:, 1]

        all_preds.append(probs.cpu().numpy())
        all_ids.extend(ids)


predictions_array = np.concatenate(all_preds)

results_df = pd.DataFrame({
    'id': all_ids,
    'prediction': predictions_array
})

print("--- Prediction Results ---")
print(results_df.head())

Generating Predictions:   0%|          | 0/6267 [00:00<?, ?it/s]

In [ ]:
results_df.to_pickle("eva02_02_preds.pkl")